Using Claude Sonnet 3.7, perform sentiment analysis on return comments

In [29]:
import pandas as pd
import duckdb
from typing import Optional, Tuple
import openpyxl

Fetch data into duckdb table

For now, CSV

In [2]:
def import_data(
        fname: str,
        clear=False,
        db_path='temp_db',
        tname='staging_table',
        ftype='csv',
        fetch_data: bool = True
        )-> Tuple[duckdb.DuckDBPyConnection, Optional[pd.DataFrame]]:
    """
    load CSV or Parquet file into a DuckDB table.
    fname = Filename of the file to load
    clear = If True, drop the table if it exists before loading, useful for reseting data.
    db_path = Name of the DuckDB database (default: 'temp_db')
    tname = Name of the table to create (default: 'staging_table')
    ftype = Type of file to load ('csv' or 'parquet', default: 'csv')
    Returns a DuckDB connection object.
    """
    con = duckdb.connect(db_path)

    if ftype not in ('csv', 'parquet'):
        con.close()
        raise ValueError("file_type must be 'csv' or 'parquet'")
    
    # handles:
    #   clearing table if clear is True
    #   create table if not exists
    #   import csv and parquet files
    func = 'read_csv_auto' if ftype == 'csv' else 'read_parquet'
    mode = 'OR REPLACE TABLE' if clear else 'TABLE IF NOT EXISTS'
    con.execute(f"""
        CREATE {mode} {tname} AS
        SELECT * FROM {func}('{fname}')
    """)

    # if fetch_data is True, return the data as a DataFrame
    df: Optional[pd.DataFrame] = None
    if fetch_data:
        df = con.execute(f"SELECT * FROM {tname}").df()

    return con, df


In [3]:
# run import_data
file_path = r'data\anthro_return_comments_200_scored.csv'
con, df = import_data(file_path, db_path='first_db', clear=True)

In [ ]:
# just fetch data
con = duckdb.connect('first_db')
df = duckdb.sql(f"SELECT * FROM staging_table").df()
con.close()

In [4]:
df.describe()


,sentiment_score,confidence_percent,confidence_lean,possible_brand_value
count,200.000000,200.000000,200.000000,200.0
mean,2.635000,79.900000,-0.026000,0.0
std,0.827769,7.861234,0.272018,0.0
min,1.000000,50.000000,-0.900000,0.0
25%,2.000000,75.000000,0.000000,0.0
50%,3.000000,75.000000,0.000000,0.0
75%,3.000000,90.000000,0.000000,0.0
max,5.000000,95.000000,0.800000,0.0


In [36]:
df.head()


,RETURN_NO,RETURN_COMMENT,sentiment_score,confidence_percent,confidence_lean,possible_brand_value,notes
0,N172323333,They arrive broken,1,95,-0.9,0,Quality failure (damaged or defective)
1,N159786387,The colors are not as vibrant as dipicted in p...,1,95,-0.9,0,Quality failure (damaged or defective)
2,N165904700,The candles came in without its own individual...,1,95,-0.9,0,Quality failure (damaged or defective)
3,CL00153609,Bracelet arrived with broken charm,1,95,-0.9,0,Quality failure (damaged or defective)
4,N169882073,Item came damaged with a dye issue on the pocket,1,95,-0.9,0,Quality failure (damaged or defective)


Connect to ChatGPT to send prompt and data for intial testing and analysis.

In [ ]:
prompt1 = f"""
You are a linguistic expert in product evaluation. Your job is to analyze customer return comments for **product feedback**, 
identify key and sub-themes related to product evaluation, and assign a sentiment score to each theme.

Requirements:
 Review the comment exclusively for *product evaluation* themes (e.g. fit, materials, quality, style).  
 Focus on the customers evaluation of the product itself.  
 Identify **1 to 4 unique themes**, each **1 to 5 words** long.  
 Avoid overly generic descriptions such as “product is bad.” Instead describe the causal themes. 
 Assign each theme a sentiment score from **1 to 5**:  
  • 1 = very negative  
  • 2 = negative  
  • 3 = neutral (no evaluative language)  
  • 4 = positive  
  • 5 = very positive  
 **Important:** purely descriptive size/fit comments without strong judgment 
   (e.g. “too small,” “sleeves too short”) → **2 (negative)**.  
 Output **only** this exact template—no shorthand numeric pairs like “1: 0” or “2: 3.”  
 For **Total_Sentiment**, output a tuple of two values:  
   1. **Average positive score** (average of all themes scored ≥4)  
   2. **Average negative score** (average of all themes scored ≤2)
   3. Use 0.0 for missing associated themes or scores.  
 e.g. `(3.5, 1.2)` or '(0.0, 4.5)'


 
START
RETURN_NO:  
Theme 1: <theme>  
Sentiment 1: <score>  
Theme 2: <theme>  
Sentiment 2: <score>  
Theme 3: <theme>  
Sentiment 3: <score>  
Theme 4: <theme>  
Sentiment 4: <score>  
Total_Sentiment: (<avg_positive>, <avg_negative>)  
END
"""

In [111]:
prompt1_test = """
You are a linguistic expert in product evaluation. You will receive a batch of return comments as a JSON array, each record with these input fields:

  • RETURN_NO  
  • RETURN_COMMENT  

You must process each record into one output row. Do not consider information from prior records for future records.
Return your answer as a single JSON array of objects, one object per input record, with exactly these keys:

  IDENTIFIER – a unique ID you generate for this row  
  RETURN_NO – copied from the input  
  RETURN_COMMENT – copied from the input  
  Theme 1 – product theme #1  
  Sentiment 1 – score for theme #1  
  Theme 2 – product theme #2 (or empty string)  
  Sentiment 2  – score for theme #2 (or 0)  
  Theme 3, Sentiment 3
  Theme 4, Sentiment 4
  Pos_mean– average of all positive scores (or 0 if no positive scores)
  Neg_mean– average of all negative scores (or 0 if no negative scores)
  Total_sentiment - "Some Positive, Some Negative", "Nuetral", "Negative", "Very Positive", "Nuetral and Negative" ... etc.

**Scoring rules (per theme):**  
1 = very negative  
2 = negative  
3 = neutral (no evaluative language)  
4 = positive  
5 = very positive  

**Size/Fit Exception**  
- Purely descriptive size/fit without judgment (“runs small,” “sleeves too short”) → **2 (negative)**  

**General rule**  
- Look for negative evaluative words (“not flattering,” “cheap feel,”, "unexpected") → score 1 or 2  
- Look for positive evaluative words (“lovely,” “beautiful color,” “soft fabric”) → score 4 or 5  

Here’s an example of the kind of JSON array you should output (for two input records):

[
  {{
    "IDENTIFIER":"A1",
    "RETURN_NO":"001",
    "RETURN_COMMENT":"The colors are not as vibrant…",
    "Theme 1":"Color accuracy",
    "Sentiment 1":2,
    "Theme 2":"Packaging damage",
    "Sentiment 2":1,
    "Theme 3":"Quality perception",
    "Sentiment 3":2,
    "Theme 4":"",
    "Sentiment 4":0,
    "Pos_mean": 0
    "Neg_mean": 1.66
    "Total_sentiment": "Negative"
  }},
  {{
    "IDENTIFIER":"A2",
    "RETURN_NO":"002",
    "RETURN_COMMENT":"LOVELY, but runs EXTREMELY large!",
    "Theme 1":"Positive style",
    "Sentiment 1":4,
    "Theme 2":"Oversize fit",
    "Sentiment 2":2,
    "Theme 3":"",
    "Sentiment 3":0,
    "Theme 4":"",
    "Sentiment 4":0,
    "Pos_mean": 4
    "Neg_mean": 2
    "Total_sentiment": "Very Positive, Some Negative"
  }}
]
"""


In [22]:
prompt2 = f"""
You are a linguistic expert in customer satisfaction evaluation. Your job is to analyze customer return comments for **customer sentiment**, identify key and sub-themes expressing a customer’s satisfaction level, and assign a sentiment score to each theme.

Requirements:
 Review the comment exclusively for *customer satisfaction* themes (e.g. “larger than expected,” “did not arrive in time,” “not hip and cool”).  
 Focus on the customers tone and overall satisfaction, **not** on product attributes.  
 Identify **1 to 4 unique themes**, each **1 to 5 words** long.  
 Avoid overly generic descriptions such as “customer is unhappy.” Instead, describe the causal experience (e.g. “late delivery,” “felt underdressed”).  
 Assign each theme a sentiment score from **1 to 5**:  
   1 = very negative  
   2 = negative  
   3 = neutral (no evaluative language)  
   4 = positive  
   5 = very positive  
 **Exception:** purely descriptive size/fit comments without judgment (e.g. “too small,” “sleeves too short”) → **3 (neutral)**.  
 **Note:** Pay special attention to customer-expression feedback around modesty or style (e.g. “too sheer,” “see-through,” “didn’t make me look good”) as **negative satisfaction** themes.  
 **Do not** output any bare numeric pairs like “1:0” or “2:3.” Use only the full field labels below.  
 For **Total_Sentiment**, output a tuple of two floats:  
   1. **Average positive score** (mean of themes scored ≥4)  
   2. **Average negative score** (mean of themes scored ≤2)  
   Use `0.0` if there are no themes in one of those groups.  
   e.g. `(3.5, 1.2)` or `(0.0, 4.5)`

Output as a JSON object with the following structure:

START  
Theme 1: <theme>  
Sentiment 1: <score>  
Theme 2: <theme>  
Sentiment 2: <score>  
Theme 3: <theme>  
Sentiment 3: <score>  
Theme 4: <theme>  
Sentiment 4: <score>  
Total_Sentiment: (<avg_positive>, <avg_negative>)  
END
"""


In [117]:
prompt2_test = """
SYSTEM: You are a linguistic expert in customer satisfaction evaluation. You will receive a JSON array of return comments, each record with:

  • RETURN_NO  
  • RETURN_COMMENT  

You must process each record into one output row. Do not consider information from prior records for future records.
Return your answer as a single JSON array of objects, one object per input record, with exactly these keys:

  IDENTIFIER       – unique ID you generate (e.g. “C1”, “C2”, …)  
  RETURN_NO        – copied from the input  
  RETURN_COMMENT   – copied from the input  
  Theme 1          – satisfaction theme #1 (1–5 words)  
  Sentiment 1      – score for theme #1 (1–5)  
  Theme 2          – satisfaction theme #2 or ""  
  Sentiment 2      – its score or 0  
  Theme 3, Sentiment 3  
  Theme 4, Sentiment 4  
  Pos_mean– average of all positive scores (or 0 if no positive scores)
  Neg_mean– average of all negative scores (or 0 if no negative scores)
  Total_sentiment - "Some Positive, Some Negative", "Nuetral", "Negative", "Very Positive", "Nuetral and Negative" ... etc.

**Scoring rules (per theme):**  
 1 = very negative  
 2 = negative  
 3 = neutral (no evaluative language)  
 4 = positive  
 5 = very positive  

**Size/Fit Exception**  
 • Purely descriptive size/fit (“runs small,” “sleeves too short”) → **3 (neutral)**  

**Focus**  
 • Only on the customer’s satisfaction and experience—**not** on product attributes.  
 • Pay special attention to style/modesty feedback (“too sheer,” “didn’t make me look good”) as negative themes.  

**Output example** (for two records):
```json
[
  {
    "IDENTIFIER":"C1",
    "RETURN_NO":"N001",
    "RETURN_COMMENT":"Dress arrived late and felt underdressed at event.",
    "Theme 1":"Late delivery",
    "Sentiment 1":2,
    "Theme 2":"Felt underdressed",
    "Sentiment 2":1,
    "Theme 3":"",
    "Sentiment 3":0,
    "Theme 4":"",
    "Sentiment 4":0,
    "Pos_mean":0.0,
    "Neg_mean":1.5,
    "Total_sentiment":"Negative"
  },
  {
    "IDENTIFIER":"C2",
    "RETURN_NO":"N002",
    "RETURN_COMMENT":"Too large but can also see the back ruffles from the front which make you look like you have gills",
    "Theme 1":"you look like have gills",
    "Sentiment 1":1,
    "Theme 2":"Too large",
    "Sentiment 2":3,
    "Theme 3":"",
    "Sentiment 3":0,
    "Theme 4":"",
    "Sentiment 4":0,
    "Pos_mean":0.0,
    "Neg_mean":1.0,
    "Total_sentiment":"Very Negative"
  }
]
"""

New Function to send prompt + df to chatGPT and evaluate.

In [71]:
import os
import json
import logging
import pandas as pd
import openai
import re
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPEN_API_KEY")
if not openai.api_key:
    raise EnvironmentError("Please set the OPEN_API_KEY environment variable")

def parse_gpt_response(response: str, debug: bool = True) -> dict:
    """
    Parse the GPT response in the custom template format into a dictionary.
    """
    # Remove START/END and split lines
    lines = response.replace('START', '').replace('END', '').strip().split('\n')
    result = {}
    for line in lines:
        line = line.strip()
        if not line:
            continue
        # Match Theme/Sentiment/Total_Sentiment
        theme_match = re.match(r"Theme (\d+): (.+)", line)
        sentiment_match = re.match(r"Sentiment (\d+): (.+)", line)
        total_match = re.match(r"Total_Sentiment: (.+)", line)
        if theme_match:
            idx, value = theme_match.groups()
            result[f"Theme {idx}"] = value.strip()
        elif sentiment_match:
            idx, value = sentiment_match.groups()
            result[f"Sentiment {idx}"] = value.strip()
        elif total_match:
            value = total_match.group(1).strip()
            result["Total_Sentiment"] = value
    if debug:
        print("Parsed single response block:", result)
    return result

def ai_format_df(prompt: str, df: pd.DataFrame, debug: bool = True) -> pd.DataFrame:
    """
    Sends `prompt` plus the JSON version of `df` to ChatGPT,
    and returns the model's JSON response as a new DataFrame.
    """
    import re
    df_json = df.to_json(orient="records")
    if debug:
        print("Prompt sent to model:\n", prompt)
    messages = [
        {"role": "system", "content": "You are a helpful assistant that takes a JSON array and returns a modified JSON array."},
        {"role": "user", "content": prompt},
        {"role": "user", "content": df_json}
    ]
    resp = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.3,
    )
    content = resp.choices[0].message.content.strip()
    if debug:
        print("Raw response from OpenAI:\n", content)
    if not content:
        raise ValueError("Empty response from OpenAI")
    logging.debug(f"AI response: {content}")

    # --- NEW: Strip code block markers if present ---
    # Remove triple backticks and optional 'json' after them
    content_clean = re.sub(r"^```(?:json)?\s*|```$", "", content.strip(), flags=re.MULTILINE).strip()
    # Remove any leading/trailing whitespace
    content_clean = content_clean.strip()

    # Try to parse as JSON, else use custom parser
    try:
        data = json.loads(content_clean)
        if debug:
            print("Parsed data from response (JSON):\n", data)
        result_df = pd.DataFrame(data)
    except json.JSONDecodeError:
        if debug:
            print("Response is not valid JSON, using custom parser.")
        blocks = [b for b in content.split('END') if b.strip()]
        parsed = [parse_gpt_response(block, debug=debug) for block in blocks]
        if debug:
            print("Parsed data from response (custom):\n", parsed)
        result_df = pd.DataFrame(parsed)
    return result_df

Function to run both product and customer sentiment analysis.

In [ ]:
print(df.iloc[190])

In [62]:
# create small sample set
set = [1,6,165,190,194]
sample_df = df.iloc[set]

In [114]:
sample_df.head()

,RETURN_NO,RETURN_COMMENT,sentiment_score,confidence_percent,confidence_lean,possible_brand_value,notes
1,N159786387,The colors are not as vibrant as dipicted in p...,1,95,-0.9,0,Quality failure (damaged or defective)
6,N171000630,Package arrived full of water with product cov...,1,95,-0.9,0,Quality failure (damaged or defective)
165,N163164258,Too large but can also see the back ruffles fr...,3,75,0.0,0,No strong sentiment detected
190,N173709893,"LOVELY, but runs EXTREMELY large! Have never ...",4,85,0.5,0,Positive tone offset by personal fit or extern...
194,N401493551,overall too large / beautiful color though,5,90,0.8,0,Strong praise with no negative sentiment


In [ ]:
sample_df = sample_df[['RETURN_NO', 'RETURN_COMMENT']].copy()
product_eval = ai_format_df(prompt1_test, sample_df, debug=False)
product_eval.head()

,IDENTIFIER,RETURN_NO,RETURN_COMMENT,Theme 1,Sentiment 1,Theme 2,Sentiment 2,Theme 3,Sentiment 3,Theme 4,Sentiment 4,Pos_mean,Neg_mean,Total_sentiment
0,A1,N159786387,The colors are not as vibrant as depicted in p...,Color accuracy,2,Quality perception,2,Design flaw,1,,0,0,1.66,Negative
1,A2,N171000630,Package arrived full of water with product cov...,Packaging damage,1,Product condition,1,,0,,0,0,1.00,Negative
2,A3,N163164258,Too large but can also see the back ruffles fr...,Fit issue,2,Design flaw,1,Appearance concern,1,,0,0,1.33,Negative
3,A4,N173709893,"LOVELY, but runs EXTREMELY large! Have never w...",Positive style,4,Fit issue,2,,0,,0,4,2.00,"Very Positive, Some Negative"
4,A5,N401493551,overall too large / beautiful color though,Fit issue,2,Color appreciation,4,,0,,0,4,2.00,"Very Positive, Some Negative"


In [116]:
results_path = r"C:\Code\URBN\review_analyzer\results\product_eval_results.csv"
product_eval.to_csv(results_path, index=False)
print(product_eval.head())


  IDENTIFIER   RETURN_NO                                     RETURN_COMMENT  \
0         A1  N159786387  The colors are not as vibrant as depicted in p...   
1         A2  N171000630  Package arrived full of water with product cov...   
2         A3  N163164258  Too large but can also see the back ruffles fr...   
3         A4  N173709893  LOVELY, but runs EXTREMELY large! Have never w...   
4         A5  N401493551         overall too large / beautiful color though   

            Theme 1  Sentiment 1             Theme 2  Sentiment 2  \
0    Color accuracy            2  Quality perception            2   
1  Packaging damage            1   Product condition            1   
2         Fit issue            2         Design flaw            1   
3    Positive style            4           Fit issue            2   
4         Fit issue            2  Color appreciation            4   

              Theme 3  Sentiment 3 Theme 4  Sentiment 4  Pos_mean  Neg_mean  \
0         Design flaw          

In [118]:
sample_df = sample_df[['RETURN_NO', 'RETURN_COMMENT']].copy()
product_eval = ai_format_df(prompt2_test, sample_df, debug=False)
product_eval.head()
results_path = r"C:\Code\URBN\review_analyzer\results\product_eval_results.csv"
product_eval.to_csv(results_path, index=False)
print(product_eval.head())


  IDENTIFIER   RETURN_NO                                     RETURN_COMMENT  \
0         C1  N159786387  The colors are not as vibrant as depicted in p...   
1         C2  N171000630  Package arrived full of water with product cov...   
2         C3  N163164258  Too large but can also see the back ruffles fr...   
3         C4  N173709893  LOVELY, but runs EXTREMELY large! Have never w...   
4         C5  N401493551         overall too large / beautiful color though   

                  Theme 1  Sentiment 1                          Theme 2  \
0  Colors not as depicted            2           Rust instead of orange   
1     Package arrived wet            2            Product cover in mold   
2               Too large            3  Back ruffles visible from front   
3    Runs extremely large            2    Can wear medium in this style   
4       Overall too large            2                  Beautiful color   

   Sentiment 2                Theme 3  Sentiment 3 Theme 4  Sentiment 4  \